In [ ]:
# Import the dependencies
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models
import tensorflow_datasets as tfds

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
(train_data, val_data), info = tfds.load('PlantVillage', split=['train[:80%]', 'train[80%:]'], with_info=True, as_supervised=True)

In [ ]:
class_names = np.array(info.features['label'].names)
print(class_names)

In [ ]:
 print("Tensorflow version " + tf.__version__)
 
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
def resize(image,label):
  image = tf.image.resize(image,[299,299])
  return (image,label)

In [ ]:
NUM_CLASSES = 38
def one_hot(image, label):
    label = tf.one_hot(label, depth=NUM_CLASSES)
    # Recasts it to Float32
    label = tf.cast(label, tf.float32)
    return image, label

In [ ]:
 
EPOCHS=10
train_sample = train_data
val_sample = val_data
 
train_data = train_data.batch(64).repeat(EPOCHS).map(resize)
train_data = train_data.map(one_hot)
val_data = val_data.batch(64).repeat(EPOCHS).map(resize)
val_data = val_data.map(one_hot)

In [ ]:
val_data

In [ ]:
# Resizing and rescaling
img_width, img_height = 299, 299
IMAGE_SIZE = (img_width, img_height)
 
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(img_width, img_height),
  layers.experimental.preprocessing.Rescaling(1./255)
])

In [ ]:
# Data augmentation 
 
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomRotation(0.4),
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomZoom(.2, .2),
  layers.experimental.preprocessing.RandomHeight(factor=[.2, .5]),
  layers.experimental.preprocessing.RandomWidth(factor=[0.2, 0.5]),
])

In [ ]:
# Transfert learning
module_selection = ("inception_v3", 299, 2048)
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/2".format(handle_base)
 
feature_extractor = hub.KerasLayer(MODULE_HANDLE, input_shape=IMAGE_SIZE+(3,),
                                   output_shape=[FV_SIZE])
do_fine_tuning = False
if do_fine_tuning:
  feature_extractor.trainable = True
  # for layer in feature_extractor.layers[-30:]:
  #   layer.trainable = True
 
else:
  feature_extractor.trainable = False

In [ ]:
model = tf.keras.Sequential([
    resize_and_rescale,
    data_augmentation,
    feature_extractor,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(38, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
 
LEARNING_RATE = 0.001
  
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

In [ ]:
# Compile the model
EPOCHS=9
LEARNING_RATE = 0.001
with tf.device('/device:GPU:0'):
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
  
  TRAIN_STEPS=tf.data.experimental.cardinality(train_sample).numpy() // 64         
  VALID_STEPS=tf.data.experimental.cardinality(val_sample).numpy() // 64
  history = model.fit(train_data,
                      validation_data=val_data,
                      epochs=EPOCHS,
                      steps_per_epoch=TRAIN_STEPS,
                      validation_steps=VALID_STEPS
                      )

In [ ]:
 print(tf.data.experimental.cardinality(train_sample).numpy() // 64)

In [ ]:
saved_model_path = "drive/My Drive/saved_leafDisease_detection"
tf.saved_model.save(model, saved_model_path)

In [ ]:
model.save("drive/My Drive/saved_leafDisease_detection")